## Задание 2. SQL  
### 2.1 Очень усердные ученики.

2.1.1 Условие

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.  

2.1.2 Задача

Дана таблица default.peas:

st_id - 	ID ученика  
timest - Время решения карточки  
correct	- Правильно ли решена горошина?  
subject	- Дисциплина, в которой находится горошина  

Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.  
NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

In [1]:
# Импортирую библиотеки

import pandas as pd
import pandahouse as ph

In [2]:
# Создаю словарь connection с нужными параметрами

connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'}

Алгоритм запроса:

1. Сначала делаю подзапрос: формирую таблицу, в которой будет просуммированно количество правильно решенных задач в разрезе месяцев 
и студентов. Для того, чтобы группировать даты помесячно, привожу исходные даты к началу месяца (у нас данные только за один месяц, но это оптимизация запроса для возможного будущего использования).
2. Из данной таблицы отбираю тех студентов, у которых количество правильно решенных задач больше или равно 20.
3. На основе полученной в подзапросе таблицы считаю кол-во уникальных id студентов    

In [3]:
# Cоздаю запрос:

q1 = '''
    SELECT COUNT(DISTINCT st_id) as number_diligent
    FROM
    (
        SELECT  toStartOfMonth(timest) as Date,
                st_id,
                SUM(correct) as Number_correct
        FROM peas
        GROUP BY Date,
                st_id
        HAVING Number_correct >= 20
    )    
    '''

In [4]:
# Отправляю запрос и записываю результат в датафрейм

diligent_students  = ph.read_clickhouse(query=q1, connection=connection_default)
diligent_students

,number_diligent
0,136


In [5]:
print(f"Вывод: количество усердных студентов - {diligent_students['number_diligent'][0]}")

Вывод: количество усердных студентов - 136


### 2.2 Оптимизация воронки

2.2.1 Условие  

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

2.2.2 Задача

Даны таблицы: 
- default.peas (см. выше), 

- default.studs:

st_id - ID ученика  
test_grp - Метка ученика в данном эксперименте

- default.final_project_check:

st_id - ID ученика  
sale_time - Время покупки  
money - Цена, по которой приобрели данный курс  
subject	- Дисциплина	  


Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

- ARPU   
- ARPAU   
- CR в покупку   
- СR активного пользователя в покупку   
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике  

ARPU считается относительно всех пользователей, попавших в группы.  
Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.  
Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

Алгоритм запроса:
1. Формирую исходную таблицу. Для этого в таблице peas считаю количество правильно решенных задач всего и по только по математике в разрезе студентов. 
2. Объединяю данную таблицу с таблицей studs через left join (отдельно проверила, что в таблице studs содержатся только уникальные значения id студентов). Левая таблица - studs, так как в ней содержатся id всех студентов, разделенных на группы, которые мне нужны для анализа. И к таблице studs  я добавляю информацию о количестве решенных задач из таблицы peas по тем студентам, по которым эта информация есть.
3. В таблице final_project_check считаю сумму оплат по всем дисциплинам и только по математике в разрезе студентов.
4. Таблицу, полученную на шаге 2, объединяю с таблицей из шага 3 с помощью left join. Левая таблица - таблица, полученная на шаге 2, так как в ней содержатся все id студентов, которые мне нужны для анализа. Исходная таблица готова.
5. На основе полученной таблицы в блоке SELECT рассчитываю все необходимые метрики  разрезе групп (control/pilot). Для отбора значений по определенным условиям использую функции sumIf и countIf.

In [6]:
# Cоздаю запрос:

q2 = '''
    SELECT test_grp,
           ROUND((SUM(money_total) / COUNT(st_id)), 2) as ARPU,
           ROUND(sumIf(money_total, correct_total > 10) / countIf(st_id, correct_total > 10), 2) as ARPAU,
           ROUND(countIf(st_id, money_total > 0) / COUNT(st_id) * 100, 2) as CR,
           ROUND(countIf(st_id, (money_total > 0 AND correct_total > 10)) / countIf(st_id, correct_total > 10) * 100, 2) as CR_active,
           ROUND(countIf(st_id, (money_math > 0 AND correct_math  >= 2)) / countIf(st_id, correct_math >= 2) * 100, 2) as CR_active_math
    FROM
    (
        SELECT *
        FROM studs as l
        LEFT JOIN 
        (
            SELECT st_id,
                   sumIf(correct, subject = 'Math') as correct_math,
                   SUM(correct) as correct_total 
            FROM peas
            GROUP BY st_id
        ) as r
        ON l.st_id = r.st_id
    ) as l
    LEFT JOIN 
    (
        SELECT st_id,
               sumIf(money, subject = 'Math') as money_math,
               SUM(money) as money_total
        FROM final_project_check
        GROUP BY st_id
    ) as r
    ON l.st_id = r.st_id
    GROUP BY test_grp
    
    '''

In [7]:
# Отправляю запрос и записываю результат в датафрейм

groups_metrics  = ph.read_clickhouse(query=q2, connection=connection_default)
groups_metrics

,test_grp,ARPU,ARPAU,CR,CR_active,CR_active_math
0,control,4540.98,10393.70,4.92,11.02,6.12
1,pilot,11508.47,29739.58,10.85,26.04,9.52


### Вывод: 
Как мы видим, все метрики в тестовой(пилотной) группе, на которой тестировали новый экран оплаты, выросли. Необходимо проверить статистическую значимость полученных различий и затем сделать вывод о том, можно ли на основе данных метрик принять решение об эффективности нового экрана оплаты.